## Interesting problems by Sergey Slotin


If time complexity isn't stated assume $O(n)$, if you're given the exact $n$ in the statement assume a TL of $1$ sec.

Tasks aren't sorted by difficulty.



## Zero Sum

You are given an array $a$ consisting of $n$ integers. Find any non-empty subset such that the sum of its elements is divisible by $n$.


<details>
  <summary>Solution</summary>
  
   Instead of a subset, let's look for a subarray with these properties. Consider the partial sums of the array modulo n, such that $s_0 = 0$ and $s_i = (a_1 + a_2 + a_3 + ... + a_i) \mod n$. Then, a subarray $(l, \ r)$ satisfies the condition if and only if $s_{l-1} = s_r$. We have $n + 1$ partial sums (including zero), however they can only take $n$ distinct values, threrefore by Pigeonhole Principle there's at least one pair $s_i = s_j$ such that $i < j$, which is the answer.

</details>




In [ ]:
def find_subset(a):
    n = len(a)
    cur = 0
    last = dict()
    last[0] = 0
    for i in range(len(a)):
        cur = (cur + a[i]) % n
        if cur in last:
            return a[last[cur] : i + 1]
        else:
            last[cur] = i + 1
            
print(find_subset([5, 9, 8, 2, 9, 4]))

## Points inside a circle

You're given $n$ points, uniformly distributed in a unit circle centered at the origin. Suggest an algorithm that sorts the points by distance from origin in $O(n)$ on average. 

<details>
  <summary>Solution</summary>
  
A similar problem with an array of uniformly distributed numbers from $0$ to $1$ can be solved by bucket sort, if you create $n$ buckets covering ranges of length $\frac{1}{n}$, only $O(1)$ numbers will land in each range on average. Here, though, there's a problem, since the expected number of points in a bucket is proportional to its area, and we can't just calculate distances, and sort points by using the same solution, as the further buckets will have way more points than the closer ones. Then let's consider splitting the circle by radii, let's set $r_0 = 0, r_n = 1$, and make all other radii satisfy $r_i ^ 2 - r_{i-1}^2 = r_{i+1}^2 - r_i^2$, this would ensure that the areas of all buckets match, this gives us the formula for $r_i = \sqrt{\frac{i}{n}}$, now we can use this to calculate the proper uniform buckets that have $O(1)$ ponits on average. 


</details>
<details>
  <summary>Frequency plot of buckets</summary>
  
![](freq_plot.png)
</details>

## Parrots

You need to transmit $64$ bytes of information. You will be assisted by $320$ parrots, each can memorize $1$ byte of information. However, even though all parrots are guaranteed to reach the destination and recite the information they were given perfectly, they may arrive in a completely different order. If all parrots are indistinguishable, how do you transmit the information?

<details>
  <summary>Solution</summary>
  
  Let's merge all the bytes into a single $512$-bit number. Now let's think about what information can be conveyed by the parrots. Realistically, nothing about order is preserved, only the set of numbers contained. Let's consider the count array of this set, the task is basically just representing the number $320$ as a sum of $256$ numbers. There are $320 + 256 - 1 \choose 256 - 1$ options, which encodes $log_2{320 + 256 - 1 \choose 256 - 1} \approx 565$ bits of information, which is clearly enough. Now, the task is to encode $k$-th option as a list of numbers and find the order of the list, which are simple combinatorial tasks. 
  
  
</details>

In [ ]:
import math
import random

def comb_rep(n, k):
    return math.comb(n + k - 1, k - 1)

def decode(n, k, lst) -> int:
    num, cur = 0, 0
    for i in range(k - 1):
        for j in range(lst[i]):
            num += comb_rep(n - cur - j, k - i - 1)
        cur += lst[i]
    return num

def encode(n, k, num) -> list:
    cur, lst = 0, []
    for i in range(k - 1):
        for j in range(n - cur + 1): 
            if comb_rep(n - cur - j, k - i - 1) <= num:
                num -= comb_rep(n - cur - j, k - i - 1)
            else:
                cur += j
                lst += [j]
                break
    lst += [n - cur]
    return lst

n, k = 320, 256
message = random.getrandbits(512)
encoded = encode(n, k, message)
parrots = [num for i, count in enumerate(encoded) for num in [i] * count]
random.shuffle(parrots)
cnt_parrots = [parrots.count(i) for i in range(256)]
print(decode(n, k, cnt_parrots) == message)


## Binary search?

Suppose there's an array of $n$ sorted uniformly distributed numbers from $0$ to $1$. Using a single query you may ask which element is at the index $i$. You should design an algorithm that looks for a number in $O(\log \log n)$ queries on average.

<details>
  <summary>Solution</summary>
  
  Clearly, we should exploit the fact that the expected position of the number $x$ is $xn$. Let's search using that guess. Now after checking the element at that position our search space is reduced to either $[1, \ xn]$ or $[xn+1, \ n]$. Now, let's recursively continue doing this. Let's analyze the runtime, too. Actually, we've just invented [interpolation search](https://en.wikipedia.org/wiki/Interpolation_search).
</details>

In [ ]:
import random
def generate(n):
    a = []
    for i in range(n):
        a.append(random.random())
    a = sorted(a)
    return a
def estimate(a):
    res = 0
    n = len(a)
    for i in range(n):
        exp = n * a[i]
        res += abs(exp - i)
    return res / n
def f(n):
    return estimate(generate(n))
i = 1
trial = 100
while i <= 1000000:
    mean = 0
    var = 0
    for j in range(trial):
        mean += f(i) / trial
        var += f(i) ** 2 / trial
    var -= mean ** 2
    print(i, mean, var, var**0.5)
    i *= 10

## Nim subsets

You're given a set $A$, consisting of $n$ integers from $0$ to $2^{32}-1$. Your task is to pick a subset $B \subseteq A$ with max sum such that there's no subset $C \subseteq B$ such that xor of $C$'s elements is $0$ in $O(n \log n)$.
<details>
  <summary>Solution</summary>
  
According to the statement, $C$ is basically supposed to be linearly independent, so clearly we're deadling with some xor basis maintenance. The condition for $C$ is obviously true because it's by definition of a basis. Then let's just sort all numbers of $A$ in decreasing order and build a basis greedily. How do we prove that this is the optimal construction, though?
</details>

## Area

You're given a unit square and $100$ circles. You need to calculate the area of the square that's covered by any of the circles with $ < 1\% $ error.
<details>
  <summary>Solution</summary>
  
  Clearly, we should exploit the fact that the expected position of the number $x$ is $xn$. Let's replace the middle point in binary search with something else using that.
</details>

## From zero to one

You're given the following code:


```python
x = 0
while x < 1:
    x += random()
```

Find the expected value of $x$.

(`random` returns a uniformly random number from $0$ to $1$)

<details>
  <summary>Solution</summary>
  
  Intuitively, you'd probably think that the answer is $1$, then you'd understand that it's probably closer to $1.5$ after doing some simulations in your head. Let's actually think about what the final value of $x$ would be. 

  
</details>

## Dominating element

You're given an array of $n$ elements. You need to answer $m$ queries, whether on the segment there's $[l, r]$ a *dominating* element — such that it appears $\frac{r-l}{2}$ times. Time complexity $O((n+m) \log n)$.

<details>
  <summary>Solution</summary>
  
If you've heard of Boyer-Moore algorithm, it's essentially just this, we can easily maintain this information in a segment tree. But I've been confused about Boyer-Moore for a long time, so let's try deriving it.

</details>

## Random binary search

You're given this code for binary search where instead of choosing the middle element, you choose a random one:

```c++
int lower_bound(int x) {
    int l = 0, r = n - 1;
    while (l < r) {
        int m = l + rand() % (r - l);
        if (t[m] >= x)
            r = m;
        else
            l = m + 1;
    }
    return l;
}
```

All keys and queries. As $n \to \infty$, what's the expected time complexity of this code (up to the constant)?

<details>
  <summary>Solution</summary>
</details>

## Min and max

You're given $68$ stones of distinct weight. Your task to find the lightest and the heaviest one using scales in $100$ operations (i.e. you pick two non-intersecting subsets and you can see whether $w1 > w2$, $w1 < w2$ or $w1 = w2$).


<details>
  <summary>Solution</summary>
  

</details>


## Permutation

You're given a permutation of size $64$. You're allowed to ask for the set of elements on a set of positions you choose, however the elements given to you will be shuffled. Restore the permutation in $6$ queries. 


<details>
  <summary>Solution</summary>
  
  It's likely that your problem solving experience would suggest that $2^6 = 64$ is a strong hint towards the solution, and it is. Let's just ask for the positions that have their i-th least significant bit as 1. 
</details>




## Geometric progression

Calculate $\frac{1-a^n}{1-a}$ for an arbitrary modulo in $O(\log n)$.

## Even cycle

You're given an undirected graph. You need to find if there's an even length simple cycle.



## Numismatist

There are $n$ greedy coin collectors, which will agree to a coin trade only if instead of a coin type they have more than one of they get a coin type they currently don't have. In total there are $k$ types of coins. You're given the number of coins of each type for every collector. Sergey wants to maximize the number of distinct coin types he has solely by trading. You should assume that only Sergey is trading coins with others, but they don't do it between themselves. 

Come up with any polynomial algorithm.

## Drunk man

A drunk man starting from the origin of number line will go to the left with probability $1-p$, and to the right with probability $p$, what's the probability (in terms of $p$) that he crosses to the left of the origin at least once?
<details>
  <summary>Solution</summary>
  
Let's think about some cases. Suppose $p < 0.5$, then as the number of steps goes towards infinity, we're guaranteed to go further and further left, as the expected change of position is negative. If $p = 0.5$, then the answer is still that we're guaranteed to cross to the left of the origin at least once, because the exptected change of position being $0$ means that we will be at position $0$ infinitely many times, and we're guaranteed to transition to the negatives at least once. For $p > 0.5$ everything is a bit more complicated. Let's denote $f_i$ the probability of going towards $-1$ at least once starting from position $i$, then.

</details>

## Meta-problem

Suppose you have a problem with 100 tests, with the input to each test being a string and the expected output being "YES" or "NO". You're also given access to a testing system, where you can submit a solution to this problem 20 times, and you're also allowed to change the code however you'd like between tests. You also have full feedback on all tests, but the verdicts are limited to "AC" and "WA", where "AC" happens if you output the correct answer and "WA" in any other case (such as RE, TL, ML and so on). Your task is to "solve" the problem.

<details>
  <summary>Solution</summary>
  
Let's think about what information we can extract from the verdicts. Let's spend our first submission on outputting "YES" to all tests, then we will know what is the expected answer for each test, if it's "AC", then the answer is "YES", otherwise it's "NO". How do we match the strings to the test numbers or to the answers? Well, now that we know the answers we can encode any arbitrary $18$ bits (not $19$ because we have to spend the last submission on actually solving the problem) of information for each string individually to somehow differentiate them from each other. Clearly, we can just use any proper polynomial hash with our $mod$ being some $18$-bit prime. By Birthday Problem you'd know that for $100$ tests a $mod$ of order $10000$ would be extremely unsafe, resulting in a couple collisions, but ours is much bigger, so we're likely fine. Nevertheless, let's still calculate the probability of collision.

The probability of all $100$ numbers from $0$ to $2^{18}-1$ being distinct is $\frac{2^{18}}{2^{18}} \times \frac{2^{18} - 1}{2^{18}} \times \frac{2^{18} - 2}{2^{18}} \times \dots \times \frac{2^{18} - 99}{2^{18}}$, which can be approximated as $\exp\left(-\frac{100^2}{2^{19}}\right)$ by using Stirling's formula, and it turns out to be a $1.89\%$ probability of failure, which is definitely enough.

</details>

## Meta-problem 2

Same as Meta-problem, but your feedback is only the number of passed tests.

100 tests, 70 submissions

<details>
  <summary>Solution</summary>
  
  </details>

## Meta-problem 3

Same as Meta-problem, but your feedback is only the number of the first test where the solution failed.

10 tests, 100 submissions

<details>
  <summary>Solution</summary>
  
</details>



## Покемоны

В турнире участвуют 1024 видов покемонов. Вы, будучи экспертом по покемонам, знаете, кто кого может победить. Иначе говоря, вам полностью известен граф-*турнир* из 1024 вершин и $1023 \times 1022 : 2$ рёбер, в котором каждые две вершины соединены ровно одним ориентированным ребром, определяющим победителя в возможном поединке. Обратите внимание, что из $a \to b$ и $b \to c$ не следует, что $a \to c$.

У вас есть 10 покеболов. Составьте команду из 10 покемонов такую, что для каждого из 1024 типов покемонов в вашей команде найдется покемон, побеждающий его. Считайте, что в битве одинаковых покемонов побеждает ваш.

## Сортировка

Можно ли отсортировать
- 5 камней за 8 взвешиваний?
- 5 камней за 7 взвешиваний?
- 20 камней за 60 взвешиваний?


## Замкнутые ломаные

Даны две замкнутые несамопересекающиеся ломаные. Определите, можно ли перевести их друг в друга с помощью параллельного переноса, поворотов и гомотетии?

## Неубывающий массив

Дан массив из $n$ целых чисел. Требуется за $2n$ операций «прибавить к одному элементу любой другой» сделать его неубывающим.



## Разрушение дерева

Дано корневое дерево. Каждую итерацию выбирается вершина (равновероятно из всех оставшихся), и удаляется всё поддерево, соответствующее этой вершине. Найти, сколько ходов в среднем будет продолжаться этот процесс, то есть матожидание номера итерации, на которой будет удалён корень дерева.




## Ниточка

В плоскую доску вбили $n$ гвоздей радиуса $r$, причём так, что соответствующие точки на плоскости образуют вершины выпуклого многоугольника. На эти гвозди натянули ниточку, причём ниточка «огибает» по кругу гвозди. Найдите длину ниточки, то есть периметр этого многоугольника с учётом закругления.

## Пельмени

Компания друзей захотела сделать заготовки для пельменей из огромного прямоугольного куска теста. Для этого в ход пошли всевозможные предметы округлой формы: стаканы, кружки, кастрюли… В итоге тесто было разделено $n$ возможно пересекающимися окружностями произвольных радиусов и центров. Нам интересно посчитать, сколько получилось заготовок, то есть на сколько кусков распалось тесто. Асимптотика $O(n^2 \log n)$.




## Окружности

Имеется окружность радиуса $R$, назовём её *внешней*. Внутри неё лежит окружность радиуса $r < R$ и соприкасается с ней. Дальше строятся бесконечное число окружностей по следующему правилу: $k$-я окружность должна

* соприкасаться с *внешней*,
* соприкасаться с предыдущей (($k-1$)-ой),
* иметь при этом максимальный радиус.

Найдите (выведите формулу за $O(1)$) радиус $k$-й такой окружности. 

## Блеф

Катя и Серёжа играют в игру. У Кати есть $n$ карт, у Серёжи — $m$. Одна дополнительная карта лежит на столе рубашкой вверх. Назовём её *особой*. Цель игроков — её отгадать. Все $n+m+1$ карт различны, игроки изначально знают только свои карты. Игроки ходят по очереди, начинает Катя. Игрок в свой ход может:

* Попытаться угадать *особую* карту. Если получилось угадать, то игрок победил, иначе проиграл. В любом случае, игра на этом заканчивается.
* Назвать любую карту из колоды. Если у соперника есть такая карта — он обязан показать ее и вывести из игры. Если же у него нет такой карты — он сообщает об этом.

С какой вероятностью выиграет Катя при оптимальной игре обоих игроков? Асимптотика $O(nm)$.



## Принцесса

В некотором королевстве жила принцесса. Она была настольно прекрасна, что каждый юноша в королевстве хотел жениться на ней. Принцесса была привередлива, поэтому твердо решила выйти замуж только за самого красивого юношу в государстве.

Она составила список из $n$ самых красивых юношей и вызывает их каждый день в случайном порядке. После того, как к ней приходит очередной юноша, она принимает решение: либо выйти за него замуж, либо нет. Если она выходит за него замуж, то она все равно просматривает всех остальных, чтобы убедиться, что он действительно самый красивый. Если это так, то они живут долго и счастливо. Если нет (если она вышла замуж не за самого красивого, либо не вышла замуж вообще), то принцесса покончит жизнь самоубийством.

У принцессы очень хорошая память, поэтому она может сравнивать красоту очередного юноши со всеми предыдущими, а также у неё тонкий вкус, поэтому никакие двое юношей не являются для неё одинаково красивыми.

Помогите принцессе разработать стратегию, которая максимизирует вероятность того, что она выйдет замуж за наиболее красивого юношу.

Асимптотика $O(n^2)$.

## Спираль

Определим спираль $(2n+1) \times (2n+1)$ как матрицу следующего вида:

$$
\begin{matrix}
21 & 22 & 23 & 24 & 25 \\
20 & 7 &  8 &  9 & 10 \\
19 & 6 &  1 &  2 & 11 \\
18 & 5 &  4 &  3 & 12 \\
17 & 16 & 15 & 14 & 13 \\
\end{matrix}
$$

Ваша задача — рассчитать ответы на $q$ запросов суммы чисел в произвольной прямоугольной области (по модулю $10^9+7$).

$q \leq 100$, $n \leq 10^9$.

## Польский лабиринт

Группа из $n$ туристов гуляет в бесконечном лабиринте. Лабиринт имеет форму треугольника Серпинского: клетка $(x, y)$ свободна, только если `x & y == 0`.

![](and_diagram.png)

Туристы устали блуждать по лабиринту и хотят встретиться в какой-нибудь свободной клетке, сумма расстояний от всех туристов до которой наименьшая. Туристы за один ход могут передвигаться вверх, вниз, влево и вправо по свободным клеткам.

$n \leq 10^5$, изначальные координаты туристов до $10^9$.



## Баланс степеней

Дан неориентированный граф. Требуется ориентировать каждое ребро так, чтобы максимизировать число вершин, у которых степень исхода равна степени захода.

## Два пути

Дан ориентированный граф. Найдите два непересекающихся по рёбрам пути из $s$ в $t$.

## Пьяница

Пьяница стоит на числовой прямой в точке 0. Каждую секунду он делает единичный шаг вправо (в сторону увеличения координат) с вероятностью $p$ и влево с вероятностью $1-p$. С какой вероятностью он когда-либо окажется в точке с отрицательной координатой?

## Иван Сусанин

Польская армия хочет добраться из поселения $s$ в поселение $t$. Ей руководят два гетмана — Камиль и Матеуш.

- Камиль руководит армией днём и водит армию по *дорогам*.
- Матеуш руководит армией ночью и совершает маневры по *секретным тропам*. 

Каждый гетман перед маршем спрашивает дорогу у Ивана Сусанина. Иван хочет задержать наступление польских войск.

Карта дорог известна Камилю. Аналогично, карта секретных троп известна Матеушу.
Поэтому Ивану не удастся их так просто обмануть — он должен каждый раз выбрать переход так, что минимальное расстояние между поселением $t$ и войском по соответствующей карте строго уменьшилось.

Вы знаете карту дорог и троп вместе с их длинами. Помогите Ивану как можно дольше (желательно, бесконечно) вести армию из $s$ в $t$.

## Варенье

В ряд стоят $n$ пустых банок из-под варенья. Вместительность $i$-й банки равна $v_i$ грамм.

Карлсон наполняет эти банки вареньем в $m$ этапов. На каждом этапе он выбирает числа $l$, $r$, $x$ и $y$, а затем пролетает над банками с $l$ по $r$, выполняя следующие операции: в банку номер $l$ он добавляет $x$ грамм варенья, в банку номер $(l + 1)$ — $(x + y)$ грамм варенья, в банку номер $(l + 2)$ — $(x + 2y)$, и так далее до $r$-той банки, в которую он положит $x + y(r - l)$ грамм варенья.

Малышу хочется определить для каждой банки наименьший номер операции, после которой она станет полной.

$n, m \leq 10^5$

## Лабиринт

Серёжа потерялся в лабиринте $n \times m$. Каждая клетка либо свободна, либо стена. Все крайние клетки — стены. Вам известно, где находится выход из лабиринта, но не известно, где находится Серёжа. Составьте для него последовательность направлений (вверх, вниз, влево, вправо) такую, после которой он окажется в клетке с выходом, вне зависимости от его стартовой позиции. Если вы прикажете ему идти в стену, то просто ничего не произойдёт.

Придумайте любой полиномиальный алгоритм.

## Обезьяна

Дана строка из $10^5$ символов латинского алфавита. Обезьяна нажимает случайные клавиши на клавиатуре (одну из 26 букв), пока не наберёт её целиком, то есть пока исходная подстрока не станет подстрокой набранной строки. Какое ожидание числа нажатых клавиш перед тем, как это произойдёт?

## Ожидание минимума

Даны $n$ случайных величин, равномерно распределенных на отрезках $[l_i, r_i]$ — у каждой величины свой отрезок. Найдите математическое ожидание минимума этих случайных величин.

Придумайте любой точный полиномиальный алгоритм.

## Шумный ксор

Загадано некое число $x$. Вы можете делать запросы следующего типа: назвать число $y$ и получить в ответ **число единичных битов** в ксор-сумме $x$, $y$ и $m$, где $m$ это случайно сгенерированная маска, в которой каждый бит имеет вероятность $p = \frac15$ быть единичным, то есть каждый бит $x \oplus y$ заменяется на противоположный с вероятностью $y$, и вам возвращается количество единичных битов. Для ясности:


```python
x = # ...

def mask(p=0.2):
    r = 0
    for i in range(32):
        if random.random() < p:
            r += 2**i
    return r

def query(y):
    return bin(x ^ y ^ mask()).count('1')
```

Ваша задача — отгадать число, используя не более 10000 попыток.

## Коммивояжер

Даны $3 \cdot 10^5$ точек на плоскости. Выберите среди них любое подмножество из 500 точек и решите для него задачу коммивояжера: найдите минимальный по длине цикл, проходящий через все эти точки.

## Анаграммы

Найдите в строке $s$ первую подстроку, являющуюся анаграммой (пререстановкой символов) строки $t$ за $O(n)$.

## Функциональный граф

Дан ориентированный граф из $n < 10^5$ вершин, в котором из каждой вершины ведет ровно одно ребро. Требуется ответить на $q < 10^5$ запросов «в какую вершину мы попадем, если начнем в вершине $v_i$ и сделаем $k_i < 10^{18}$ переходов» за время $O(q + n)$.

## Асинхронная шляпа

Серёжа и его $(n - 1)$ друзей решили поиграть в «шляпу», в которой один игрок должен за ограниченное время объяснить как можно больше слов, чтобы его партнер их отгадал.

Каждый игрок должен пообщаться с любым другим по разу; обычно игра проводится так:

- 1-й игрок объясняет в течение минуты слова 2-му,
- 2-й игрок объясняет слова 3-му,
- ...,
- $n$-й игрок объясняет слова 1-му,
- 1-й игрок объясняет слова 3-му,
- 2-й игрок объясняет слова 4-му…

…и так далее, пока $(n-1)$-й игрок не закончит объяснять слова $(n-2)$-ому.

Если друзей собралось много, то игра может занять приличное время. Серёжу интересует, какое минимальное время она может длиться, если разрешить парам участников общаться между собой одновременно и в любом порядке.

Для данного $n \le 500$, найдите минимальное количество времени $k$ и соответствующее ему расписание.

## Random coffee

В компании, в которой вы работаете, устроено неизвестное число людей — от одного до бесконечности с равной вероятностью. Для борьбы с одиночеством, каждый сотрудник участвует в «random coffee»: каждую неделю вы встречаетесь со случайным человеком из компании, чтобы попить кофе и обсудить что угодно.

Вы участвовали в random coffee $n$ раз и пообщались с $k$ разными людьми (с некоторыми — более одного раза). Какое наиболее вероятное число человек работает в компании?

## Мафия

В «мафию» играют 13 человек, из которых 10 мирных и 3 мафии. Все роли розданы с помощью стандартной колоды игральных карт: заранее выбрали и перемешали 10 красных и 3 чёрные карты, кто вытянул черную — мафия. Все карты различны и известны всем. Игра начинается с дневного голосования.

Как мирным гарантированно победить?

